# Aula 30 – Generative Adversarial Networks

## 1. Introdução

As Generative Adversarial Networks (Redes adversárias gerativas) ou GANs, são a tecnologia por trás da técnica conhecida por Deep Fake, onde imagens que nunca existiram são geradas, geralmente utilizadas para trocar personagens em imagens. Não seria possível esgotar o assunto em uma aula, então o objetivo aqui é mostrar o conceito e indicar caminhos para quem quiser depois conhecer melhor por conta própria.

A ideia por trás das GANs é colocar duas redes convolucionais, G (geradora) e D (discriminadora), para competirem no processo de treinamento, sendo que G gera a imagem, enquanto D classifica a imagem. O objetivo é que G engane D e que suas imagens geradas sejam classificadas como imagens de objetos reais que D é capaz de reconhecer. É importante enfatizar que ambas treinam, isto é, se D é enganada, essa informação é utilizada para melhorar seus parâmetros para que ela consiga reconhecer melhor imagens geradas, dificultando, então, cada vez mais, o trabalho de G, que precisa se desenvolver para gerar imagens cada vez melhores.

O outro ponto importante aqui é como fazer uma rede convolucional gerar, e não classificar imagens. Bom, a ideia é inverter o fluxo de informações, isto é, os neurônios que seriam de “saída”, ou seja, os que classificariam, recebem a entrada, ou seja, a informação de qual objeto queremos que seja gerado. Os sinais percorrem a rede até a entrada, onde seria dada a imagem, ou seja, seus pixels, informando as supostas intensidades de pixels que deveriam ser dadas de entrada para a classificação desejada. Para ser mais preciso, isso é reconhecido como uma rede “deconvolucional”.

Basicamente, a imagem gerada é uma combinação das características aprendidas para identificar uma imagem. GANs rudimentares geram resultados que enganam pessoas em uma passagem rápida pelas imagens, pois as características esperadas estão lá. Porém, muitas vezes, parecem quebra-cabeças mal montados (peças corretas, mas nos cantos errados), quando observamos melhor. Entretanto, o avanço da técnica está permitindo a geração de imagens cada vez mais difíceis de se identificar como falsas. A principal característica necessária para a geração de boas GANs é um treinamento intensivo, com uma base muito grande de exemplos, sendo computacionalmente pesado o processo de gerar, embora executar não seja tão exigente.

## 2. Material Complementar

http://www.iangoodfellow.com/slides/2016-12-9-gans.pdf

https://www.youtube.com/watch?v=RvgYvHyT15E

## 3. Exercícios

- Realize o seguinte tutorial: 
https://towardsdatascience.com/build-a-super-simple-gan-in-pytorch-54ba349920e4


In [33]:
#Utils.py
from typing import List, Tuple
import numpy as np
import math


def create_binary_list_from_int(number: int) -> List[int]:
    """Creates a list of the binary representation of a positive integer
    Args:
        number: An integer
    Returns:
        The binary representation of the provided positive integer number as a list.
    """
    if number < 0 or type(number) is not int:
        raise ValueError("Only Positive integers are allowed")

    return [int(x) for x in list(bin(number))[2:]]


def generate_even_data(
    max_int: int, batch_size: int = 16
) -> Tuple[List[int], List[List[int]]]:
    """An infinite data generator which yields
    Args:
        max_int: The maximum input integer value
        batch_size: The size of the training batch.
    Returns:
        A Tuple with the labels and the input data.
        labels:
        data:
    """

    # Get the number of binary places needed to represent the maximum number
    max_length = int(math.log(max_int, 2))

    # Sample batch_size number of integers in range 0-max_int
    sampled_integers = np.random.randint(0, int(max_int / 2), batch_size)

    # create a list of labels all ones because all numbers are even
    labels = [1] * batch_size

    # Generate a list of binary numbers for training.
    data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]
    data = [([0] * (max_length - len(x))) + x for x in data]

    return labels, data


def convert_float_matrix_to_int_list(
    float_matrix: np.array, threshold: float = 0.5
) -> List[int]:
    """Converts generated output in binary list form to a list of integers
    Args:
        float_matrix: A matrix of values between 0 and 1 which we want to threshold and convert to
            integers
        threshold: The cutoff value for 0 and 1 thresholding.
    Returns:
        A list of integers.
    """
    return [
        int("".join([str(int(y)) for y in x]), 2) for x in float_matrix >= threshold
    ]

In [34]:
#Models.py
import torch.nn as nn


class Generator(nn.Module):
    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense_layer(x))


class Discriminator(nn.Module):
    def __init__(self, input_length: int):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))

In [35]:
#Train.py
from typing import Tuple
import math

import torch
import torch.nn as nn

#from models import Discriminator, Generator
#from utils import generate_even_data, convert_float_matrix_to_int_list


def train(
    max_int: int = 128,
    batch_size: int = 16,
    training_steps: int = 500,
    learning_rate: float = 0.001,
    print_output_every_n_steps: int = 10,
) -> Tuple[nn.Module]:
    """Trains the even GAN
    Args:
        max_int: The maximum integer our dataset goes to.  It is used to set the size of the binary
            lists
        batch_size: The number of examples in a training batch
        training_steps: The number of steps to train on.
        learning_rate: The learning rate for the generator and discriminator
        print_output_every_n_steps: The number of training steps before we print generated output
    Returns:
        generator: The trained generator model
        discriminator: The trained discriminator model
    """
    input_length = int(math.log(max_int, 2))

    # Models
    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    # Optimizers
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)
    discriminator_optimizer = torch.optim.Adam(
        discriminator.parameters(), lr=learning_rate
    )

    # loss
    loss = nn.BCELoss()

    for i in range(training_steps):
        # zero the gradients on each iteration
        generator_optimizer.zero_grad()

        # Create noisy input for generator
        # Need float type instead of int
        noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
        generated_data = generator(noise)

        # Generate examples of even real data
        true_labels, true_data = generate_even_data(max_int, batch_size=batch_size)
        true_labels = torch.tensor(true_labels).float()
        true_data = torch.tensor(true_data).float()

        # Train the generator
        # We invert the labels here and don't train the discriminator because we want the generator
        # to make things the discriminator classifies as true.
        generator_discriminator_out = discriminator(generated_data)
        generator_loss = loss(generator_discriminator_out, true_labels)
        generator_loss.backward()
        generator_optimizer.step()

        # Train the discriminator on the true/generated data
        discriminator_optimizer.zero_grad()
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss(true_discriminator_out, true_labels)

        # add .detach() here think about this
        generator_discriminator_out = discriminator(generated_data.detach())
        generator_discriminator_loss = loss(
            generator_discriminator_out, torch.zeros(batch_size)
        )
        discriminator_loss = (
            true_discriminator_loss + generator_discriminator_loss
        ) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()
        if i % print_output_every_n_steps == 0:
            print(convert_float_matrix_to_int_list(generated_data))

    return generator, discriminator


if __name__ == "__main__":
    train()

ValueError: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])) is deprecated. Please ensure they have the same size.

In [18]:
#TrainTest.py
import math
from typing import List
import unittest

from ddt import data, ddt, unpack
import torch

#from utils import convert_float_matrix_to_int_list
#from train import train


@ddt
class TrainTest(unittest.TestCase):
    @data(
        (128, 16, 500, 0.001, "Test reasonable parameters"),
        (256, 16, 500, 0.001, "Test reasonable parameters"),
    )
    @unpack
    def test_train(
        self,
        max_int: int,
        batch_size: int,
        training_steps: int,
        learning_rate: float,
        test_description: str,
    ):
        input_length = int(math.log(max_int, 2))
        generator, discriminator = train(
            max_int=max_int,
            batch_size=batch_size,
            training_steps=training_steps,
            learning_rate=learning_rate,
            print_output_every_n_steps=1000000,
        )
        noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
        generated_data = generator(noise)
        for num in convert_float_matrix_to_int_list(generated_data):
            self.assertEqual(num % 2, 0, test_description)


if __name__ == "__main__":
    unittest.main()

E
ERROR: /home/vando/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/vando/'

----------------------------------------------------------------------
Ran 1 test in 0.007s

FAILED (errors=1)


SystemExit: True

/home/vando/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [32]:
#UtilTest.py
import math
from typing import List
import unittest

from ddt import data, ddt, unpack
import numpy as np

# from utils import (
#     create_binary_list_from_int,
#     generate_even_data,
#     convert_float_matrix_to_int_list,
# )


@ddt
class UtilsTest(unittest.TestCase):
    @data(
        (1, [1], "test single odd"),
        (0, [0], "test zero"),
        (7, [1, 1, 1], "test small prime"),
        (128, [1, 0, 0, 0, 0, 0, 0, 0], "test large even"),
        (129, [1, 0, 0, 0, 0, 0, 0, 1], "test large odd"),
    )
    @unpack
    def test_create_binary_list_from_int(
        self, number: int, expected: List[int], test_description: str
    ):
        output = create_binary_list_from_int(number)
        self.assertListEqual(expected, output, test_description)

    @data(
        (2, 2, "test small input"),
        (128, 2, "test standard input"),
        (128, 16, "test standard input big batch size"),
        (1024, 16, "test big input"),
    )
    @unpack
    def test_generate_even_data(
        self, max_int: int, batch_size: int, test_description: str
    ):
        labels, output = generate_even_data(max_int, batch_size=batch_size)
        self.assertEqual(len(labels), batch_size)
        self.assertEqual(len(output), batch_size)
        for binary_num in output:
            self.assertEqual(binary_num[-1], 0, test_description)
            self.assertEqual(len(binary_num), math.log(max_int, 2), test_description)

    @data(([[0.6, 0.2], [0.5, 0.5]], [2, 3], "test two values"))
    @unpack
    def test_convert_float_matrix_to_int_list(
        self, matrix: List[List[float]], expected: List[int], test_description: str
    ):
        matrix = np.array(matrix)
        output = convert_float_matrix_to_int_list(np.array(matrix))
        self.assertListEqual(expected, output, test_description)


if __name__ == "__main__":
    unittest.main()

E
ERROR: /home/vando/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/vando/'

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (errors=1)


SystemExit: True

/home/vando/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
